In [1]:
#Importamos las librerias a usar
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import cross_val_score

In [2]:
#Leemos nuestros DataFrames a utilizar
df = pd.read_csv('Pokemon.csv')
combats = pd.read_csv('Combat1.csv')
test = pd.read_csv('tests.csv')

In [3]:
#Reasignamos el nombre a la columna # para facil manipulacion
df.rename(columns={'#':'Num'}, inplace=True)

In [4]:
#Re Asignamos el index para el correcto manejo de la base
df.set_index(df.Num, inplace=True)

In [5]:
#Creamos un nuevo DataSet con las columnas que vamos a usar
poke = df.filter(["Name","HP","Attack","Defense","Sp.Atk","Sp.Def","Speed"], axis=1)
poke

Name   HP  Attack  Defense  Sp.Atk  Sp.Def  Speed
Num                                                          
1      Bulbasaur   45      49       49      65      65     45
2        Ivysaur   60      62       63      80      80     60
3       Venusaur   80      82       83     100     100     80
4     Charmander   39      52       43      60      50     65
5     Charmeleon   58      64       58      80      65     80
6      Charizard   78      84       78     109      85    100
7       Squirtle   44      48       65      50      64     43
8      Wartortle   59      63       80      65      80     58
9      Blastoise   79      83      100      85     105     78
10      Caterpie   45      30       35      20      20     45
11       Metapod   50      20       55      25      25     30
12    Butterfree   60      45       50      90      80     70
13        Weedle   40      35       30      20      20     50
14        Kakuna   45      25       50      25      25     35
15      Beedrill   65      90       40      45      80     75
16        Pidgey   40      45       40      35      35     56
17     Pidgeotto   63      60       55      50      50     71
18       Pidgeot   83      80       75      70      70    101
19       Rattata   30      56       35      25      35     72
20      Raticate   55      81       60      50      70     97
21       Spearow   40      60       30      31      31     70
22        Fearow   65      90       65      61      61    100
23         Ekans   35      60       44      40      54     55
24         Arbok   60      85       69      65      79     80
25       Pikachu   35      55       40      50      50     90
26        Raichu   60      90       55      90      80    110
27     Sandshrew   50      75       85      20      30     40
28     Sandslash   75     100      110      45      55     65
29      NidoranF   55      47       52      40      40     41
30      Nidorina   70      62       67      55      55     56
..           ...  ...     ...      ...     ...     ...    ...
780       Drampa   78      60       85     135      91     36
781     Dhelmise   70     131      100      86      90     40
782     Jangmo-o   45      55       65      45      45     45
783     Hakamo-o   55      75       90      65      70     65
784      Kommo-o   75     110      125     100     105     85
785    Tapu Koko   70     115       85      95      75    130
786    Tapu Lele   70      85       75     130     115     95
787    Tapu Bulu   70     130      115      85      95     75
788    Tapu Fini   70      75      115      95     130     85
789       Cosmog   43      29       31      29      31     37
790      Cosmoem   43      29      131      29     131     37
791     Solgaleo  137     137      107     113      89     97
792       Lunala  137     113       89     137     107     97
793     Nihilego  109      53       47     127     131    103
794     Buzzwole  107     139      139      53      53     79
795    Pheromosa   71     137       37     137      37    151
796    Xurkitree   83      89       71     173      71     83
797   Celesteela   97     101      103     107     101     61
798      Kartana   59     181      131      59      31    109
799     Guzzlord  223     101       53      97      53     43
800     Necrozma   97     107      101     127      89     79
801     Magearna   80      95      115     130     115     65
802    Marshadow   90     125       80      90      90    125
803      Poipole   67      73       67      73      67     73
804    Naganadel   73      73       73     127      73    121
805    Stakataka   61     131      211      53     101     13
806  Blacephalon   53     127       53     151      79    107
807      Zeraora   88     112       75     102      80    143
808       Meltan   46      65       65      55      35     34
809     Melmetal  135     143      143      80      65     34

[809 rows x 7 columns]

In [6]:
#Remplazamos los numeros con el nombre del pokemon correspondiente
cols = ["First_pokemon","Second_pokemon","Winner"]
fights = combats[cols].replace(df.Name)
fights.head()

First_pokemon Second_pokemon     Winner
0      Larvitar        Nuzleaf    Nuzleaf
1      Virizion      Terrakion  Terrakion
2       Togetic       Beheeyem   Beheeyem
3        Slugma      Druddigon  Druddigon
4       Omastar        Shuckle    Omastar

In [22]:
#Transformamos en boleano nuestro resultado del combate
combats.Winner[combats.Winner == combats.First_pokemon] = 0
combats.Winner[combats.Winner == combats.Second_pokemon] = 1

In [29]:
#Creamos una funcion de normalizacion para los datos que usaremos
#Fue robada de internet y modificada por su servidor
def normalization(data_df):
    stats=["HP","Attack","Defense","Sp.Atk","Sp.Def","Speed"]
    stats_df=poke[stats].T.to_dict("list")
    one=data_df.First_pokemon.map(stats_df)
    two=data_df.Second_pokemon.map(stats_df)
    temp_list=[]
    for i in range(len(one)):
        temp_list.append(np.array(one[i])-np.array(two[i]))
    new_test = pd.DataFrame(temp_list, columns=stats)
    for c in stats:
        description=new_test[c].describe()
        new_test[c]=(new_test[c]-description['min'])/(description['max']-description['min'])
    return new_test

In [30]:
#Aplicamos la funcion y creamos un nuevo DataSet
#Concatenamos las dos tablas
data = normalization(combats)
data = pd.concat([data, combats.Winner], axis=1)

In [31]:
#Usamos la columna Winner para hacer nuestro testeo
X=data.drop("Winner",axis=1)
y=data["Winner"]

In [32]:
#Definimos el train y asignamos el size
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [36]:
#Aplicamos modelo de ML para predecir (en el testeo original se usaron 5 y este fue el que mas me agrado)
clf_rfc = RandomForestClassifier(n_estimators=50)
clf_rfc.fit(X_train, y_train)
accuracies = cross_val_score(estimator = clf_rfc, X = X_train, y = y_train, cv = 5,verbose = 1)
y_pred = clf_rfc.predict(X_test)

print('')
print('####### RandomForestClassifier #######')
print('Score : %.4f' % clf_rfc.score(X_test, y_test))
print(accuracies)

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred)**0.5
r2 = r2_score(y_test, y_pred)

print('')
print('MSE    : %0.2f ' % mse)
print('MAE    : %0.2f ' % mae)
print('RMSE   : %0.2f ' % rmse)
print('R2     : %0.2f ' % r2)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.



####### RandomForestClassifier #######
Score : 0.9468
[0.95495634 0.94959399 0.94406987 0.95050567 0.94759424]

MSE    : 0.05 
MAE    : 0.05 
RMSE   : 0.23 
R2     : 0.79 


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    5.7s finished


In [34]:
#Aplicamos modelo de ML a nuestro test para predecir las victorias
battle=normalization(test)
pred=clf_rfc.predict(battle)
test["Winner"]=[test["First_pokemon"][i] if pred[i]==0 else test["Second_pokemon"][i] for i in range(len(pred))]

In [35]:
#Remplazamos los numeros por nombres eh imprimimos los resultados de las batallas
combats_name = test[cols].replace(poke.Name)
combats_name.head()

First_pokemon Second_pokemon      Winner
0      Magikarp         Seadra      Seadra
1     Diggersby       Qwilfish    Qwilfish
2        Goodra     Kangaskhan  Kangaskhan
3      Quagsire       Stunfisk    Quagsire
4     Sandshrew        Froakie     Froakie